# NARDL-Fourier Library: Complete Example Notebook

**Author:** Dr. Merwan Roudane  
**Email:** merwanroudane920@gmail.com  
**GitHub:** https://github.com/merwanroudane/fnardl

This notebook demonstrates all features of the `nardl-fourier` library:
1. Standard NARDL (Shin et al., 2014)
2. Fourier NARDL (Zaghdoudi et al., 2023)
3. Bootstrap NARDL (Bertelli et al., 2022)

## Setup and Installation

In [ ]:
# Install the package (run once)
# !pip install -e .

import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import nardl-fourier
from nardl_fourier import NARDL, FourierNARDL, BootstrapNARDL
from nardl_fourier.output import ResultsTable, NARDLPlots
from nardl_fourier.diagnostics import run_all_diagnostics
from nardl_fourier.utils import check_stationarity, partial_sum_decomposition

print("✓ Library loaded successfully!")

## Load Sample Data

We use the Kilian (2009) oil price data for demonstration.

In [ ]:
# Load data
try:
    data = pd.read_excel('../nardl_fourier/data/killian.xlsx')
except:
    # Generate sample data if file not found
    np.random.seed(42)
    n = 120
    
    # Generate variables
    oil = np.cumsum(np.random.normal(0, 1, n)) + 50
    gdp = np.cumsum(np.random.normal(0.02, 0.5, n)) + 100
    gdp2 = gdp ** 2 / 1000
    coal = 0.3 * oil + 0.5 * gdp - 0.02 * gdp2 + np.cumsum(np.random.normal(0, 0.5, n))
    
    data = pd.DataFrame({
        'coal': coal,
        'oil': oil,
        'gdp': gdp,
        'gdp2': gdp2
    })

print(f"Data shape: {data.shape}")
data.head(10)

In [ ]:
# Descriptive statistics
data.describe().round(4)

In [ ]:
# Plot the data
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for ax, col in zip(axes.flatten(), data.columns):
    ax.plot(data[col], linewidth=1.5)
    ax.set_title(col, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('Time Series Data', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Unit Root Testing

In [ ]:
# Check stationarity for all variables
print("="*60)
print("UNIT ROOT TESTS (ADF)")
print("="*60)

for col in data.columns:
    result = check_stationarity(data[col], method='adf')
    print(f"\n{col}:")
    print(f"  Level: ADF = {result['statistic']:.4f}, p = {result['p_value']:.4f} → {result['conclusion']}")
    
    # Test first difference
    result_diff = check_stationarity(data[col].diff().dropna(), method='adf')
    print(f"  Δ{col}: ADF = {result_diff['statistic']:.4f}, p = {result_diff['p_value']:.4f} → {result_diff['conclusion']}")

## Partial Sum Decomposition

The NARDL model decomposes x into positive and negative partial sums:
- $x_t^+ = \sum_{j=1}^{t} \max(\Delta x_j, 0)$
- $x_t^- = \sum_{j=1}^{t} \min(\Delta x_j, 0)$

In [ ]:
# Demonstrate partial sum decomposition
oil_pos, oil_neg = partial_sum_decomposition(data['oil'])

fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

axes[0].plot(data['oil'], 'b-', linewidth=1.5, label='Oil Price')
axes[0].set_ylabel('Oil Price')
axes[0].set_title('Original Series', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(oil_pos, 'g-', linewidth=1.5, label='Oil⁺ (Positive)')
axes[1].set_ylabel('Cumulative')
axes[1].set_title('Positive Partial Sum (Increases)', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(oil_neg, 'r-', linewidth=1.5, label='Oil⁻ (Negative)')
axes[2].set_ylabel('Cumulative')
axes[2].set_xlabel('Time')
axes[2].set_title('Negative Partial Sum (Decreases)', fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 1. Standard NARDL Model

Based on Shin, Yu & Greenwood-Nimmo (2014)

In [ ]:
# Fit Standard NARDL
nardl = NARDL(
    data=data,
    depvar='coal',
    exog_vars=['gdp', 'gdp2'],
    decomp_vars=['oil'],
    maxlag=4,
    ic='AIC',
    case=3
)

print("✓ NARDL model estimated successfully!")

In [ ]:
# Model Summary
print(nardl.summary())

In [ ]:
# Long-run multipliers table
print("\nLONG-RUN MULTIPLIERS")
print("="*60)
nardl.long_run_table()

In [ ]:
# Short-run coefficients table
print("\nSHORT-RUN COEFFICIENTS")
print("="*60)
nardl.short_run_table()

In [ ]:
# Bounds test for cointegration
print("\nPSS BOUNDS TEST")
print("="*60)
bt = nardl.bounds_test
print(f"F-statistic: {bt['f_statistic']:.4f}")
print(f"\nCritical Values (Case {bt['case']}):")
for sig in ['10%', '5%', '1%']:
    cv = bt['critical_values']['F'][sig]
    print(f"  {sig}: I(0) = {cv['I(0)']:.4f}, I(1) = {cv['I(1)']:.4f}")
print(f"\nDecision (5%): {bt['decision']['F_5%']}")

In [ ]:
# Dynamic Multipliers Plot
fig = nardl.plot_multipliers('oil')
plt.show()

In [ ]:
# CUSUM Stability Test
fig = nardl.plot_cusum()
plt.show()

In [ ]:
# Diagnostics
print("\nDIAGNOSTIC TESTS")
print("="*60)
nardl.diagnostics_table()

---
# 2. Fourier NARDL Model

Based on Zaghdoudi et al. (2023) - captures smooth structural breaks

In [ ]:
# Fit Fourier NARDL
fnardl = FourierNARDL(
    data=data,
    depvar='coal',
    exog_vars=['gdp', 'gdp2'],
    decomp_vars=['oil'],
    maxlag=4,
    max_freq=3,
    ic='AIC',
    case=3
)

print("✓ Fourier NARDL model estimated!")
print(f"\nOptimal Fourier Frequency: k* = {fnardl.best_freq}")

In [ ]:
# Model Summary
print(fnardl.summary())

In [ ]:
# Plot Fourier terms
fig = fnardl.plot_fourier_terms()
plt.show()

In [ ]:
# Fourier term significance test
print("\nFOURIER SIGNIFICANCE TEST")
print("="*60)
ft = fnardl.fourier_test
print(f"F-statistic (sin, cos = 0): {ft['f_statistic']:.4f}")
print(f"p-value: {ft['p_value']:.4f}")
print(f"Significant: {'Yes' if ft['significant'] else 'No'}")

In [ ]:
# Compare NARDL vs Fourier NARDL
print("\nMODEL COMPARISON")
print("="*60)
print(f"{'Metric':<25} {'NARDL':>15} {'Fourier NARDL':>15}")
print("-"*60)
print(f"{'R-squared':<25} {nardl.model.rsquared:>15.4f} {fnardl.model.rsquared:>15.4f}")
print(f"{'Adj. R-squared':<25} {nardl.model.rsquared_adj:>15.4f} {fnardl.model.rsquared_adj:>15.4f}")
print(f"{'AIC':<25} {nardl.best_ic:>15.4f} {fnardl.best_ic:>15.4f}")

---
# 3. Bootstrap NARDL Model

Based on Bertelli, Vacca & Zoia (2022) - eliminates inconclusive zones

In [ ]:
# Fit Bootstrap NARDL
bnardl = BootstrapNARDL(
    data=data,
    depvar='coal',
    exog_vars=['gdp', 'gdp2'],
    decomp_vars=['oil'],
    maxlag=4,
    max_freq=3,
    n_bootstrap=1000,
    random_state=42
)

print("✓ Bootstrap NARDL model estimated!")

In [ ]:
# Bootstrap Cointegration Decision
print(bnardl.cointegration_decision())

In [ ]:
# Full Summary
print(bnardl.summary())

In [ ]:
# Bootstrap Test Results Table
if bnardl.bootstrap_test:
    bnardl.bootstrap_test.summary_table()

In [ ]:
# Plot Bootstrap Distributions
fig = bnardl.plot_bootstrap_distributions()
plt.show()

---
# 4. Publication-Ready Tables

In [ ]:
# Create results table object
table = ResultsTable(bnardl)

# Regression results
print("\nREGRESSION RESULTS")
table.regression_table()

In [ ]:
# Wald test for asymmetry
print("\nWALD TESTS FOR ASYMMETRY")
table.wald_test_table()

In [ ]:
# Export to LaTeX
# table.to_latex('nardl_results.tex')
# print("✓ Results exported to LaTeX")

---
# 5. Comprehensive Diagnostics

In [ ]:
# Run all diagnostics
diag = run_all_diagnostics(bnardl)

print("DIAGNOSTIC SUMMARY")
print("="*60)
print(f"All tests passed: {diag['summary']['all_tests_passed']}")
if diag['summary']['issues_detected']:
    print(f"Issues: {', '.join(diag['summary']['issues_detected'])}")

In [ ]:
# Detailed diagnostics table
from nardl_fourier.diagnostics.tests import diagnostics_summary_table
diagnostics_summary_table(diag)

In [ ]:
# Residual diagnostics plots
plots = NARDLPlots(bnardl)
fig = plots.residuals()
plt.show()

---
# 6. Interpretation Guide

## Key Results to Report

1. **Bounds Test**: F-statistic vs critical values
2. **ECT (ρ)**: Should be negative and significant (-2 < ρ < 0)
3. **Long-run multipliers**: L⁺ and L⁻ with significance
4. **Asymmetry test**: Wald test p-value < 0.05 indicates asymmetry
5. **Half-life**: Time to absorb 50% of shock

## Example Interpretation

> The bounds test F-statistic (X.XX) exceeds the upper I(1) bound at the 5% level,
> confirming a long-run cointegrating relationship. The error correction term
> (ρ = -0.XX) is negative and significant, with a half-life of X.X periods.
> The long-run positive multiplier (L⁺ = X.XX) differs significantly from the
> negative multiplier (L⁻ = X.XX), as confirmed by the Wald test (F = X.XX, p < 0.05),
> indicating an asymmetric long-run relationship.

---
## References

1. Shin, Y., Yu, B., & Greenwood-Nimmo, M. (2014). *Festschrift in Honor of Peter Schmidt*
2. Pesaran, M. H., Shin, Y., & Smith, R. J. (2001). *Journal of Applied Econometrics*
3. Zaghdoudi, T. et al. (2023). *Energy*, 285, 129416
4. Bertelli, S., Vacca, G., & Zoia, M. (2022). *Economic Modelling*, 116, 105987